# Description<br>
Initially I took this up to see how good to see how well I can perform on this data set using only the words as features and nothing else. Using much smaller neural networks than used here and tuning them i got an f1 score of 0.92. Then I tried doing the same with including other models like neural netwoks. The maximum f1 score that i have got is <strong>0.9764309764309764</strong>. This score was obtained after Ensembling the models using a voting classifier

 Since no voting classifier is present in Keras I built an approximate version of my own.

In [1]:
import tensorflow as tf
import numpy 
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.layers import Dropout
%matplotlib inline

C:\Users\Lenovo\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pandas as pd
data = pd.read_csv('sms-spam.csv',usecols=['is spam','text'],names=['is spam','text'],encoding='latin1')
data = data.iloc[1:,:]
data['is spam'] = data['is spam'].map({'spam':1,'ham':0})
data.head()

,is spam,text
1,0,"Go until jurong point, crazy.. Available only ..."
2,0,Ok lar... Joking wif u oni...
3,1,Free entry in 2 a wkly comp to win FA Cup fina...
4,0,U dun say so early hor... U c already then say...
5,0,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
def percentage_digits(text):
    counter = 0
    for i in text:
        if i.isdigit():
            counter += 1
    return (counter / len(text)) * 100

data['percentage digits'] = data['text'].apply(percentage_digits)
data.head()

,is spam,text,percentage digits
1,0,"Go until jurong point, crazy.. Available only ...",0.000000
2,0,Ok lar... Joking wif u oni...,0.000000
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,16.129032
4,0,U dun say so early hor... U c already then say...,0.000000
5,0,"Nah I don't think he goes to usf, he lives aro...",0.000000


In [4]:
def percentage_question_mark(text):
    counter = 0
    for i in text:
        if i == '?':
            counter += 1
    return (counter / len(text)) * 100

data['percentage question mark'] = data['text'].apply(percentage_question_mark)
data.head()

,is spam,text,percentage digits,percentage question mark
1,0,"Go until jurong point, crazy.. Available only ...",0.000000,0.0
2,0,Ok lar... Joking wif u oni...,0.000000,0.0
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,16.129032,0.0
4,0,U dun say so early hor... U c already then say...,0.000000,0.0
5,0,"Nah I don't think he goes to usf, he lives aro...",0.000000,0.0


In [5]:
def percentage_exclamation_mark(text):
    counter = 0
    for i in text:
        if i == '!':
            counter += 1
    return (counter / len(text)) * 100

data['percentage exclamation mark'] = data['text'].apply(percentage_exclamation_mark)
data.head()

,is spam,text,percentage digits,percentage question mark,percentage exclamation mark
1,0,"Go until jurong point, crazy.. Available only ...",0.000000,0.0,0.0
2,0,Ok lar... Joking wif u oni...,0.000000,0.0,0.0
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,16.129032,0.0,0.0
4,0,U dun say so early hor... U c already then say...,0.000000,0.0,0.0
5,0,"Nah I don't think he goes to usf, he lives aro...",0.000000,0.0,0.0


In [6]:
def percentage_caps(text):
    counter = 0
    for i in text:
        if i.isupper():
            counter += 1
    return (counter / len(text)) * 100

data['percentage caps'] = data['text'].apply(percentage_caps)
data.head()

,is spam,text,percentage digits,percentage question mark,percentage exclamation mark,percentage caps
1,0,"Go until jurong point, crazy.. Available only ...",0.000000,0.0,0.0,2.702703
2,0,Ok lar... Joking wif u oni...,0.000000,0.0,0.0,6.896552
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,16.129032,0.0,0.0,6.451613
4,0,U dun say so early hor... U c already then say...,0.000000,0.0,0.0,4.081633
5,0,"Nah I don't think he goes to usf, he lives aro...",0.000000,0.0,0.0,3.278689


In [7]:
data['length'] = data['text'].apply(len)
data.head()

,is spam,text,percentage digits,percentage question mark,percentage exclamation mark,percentage caps,length
1,0,"Go until jurong point, crazy.. Available only ...",0.000000,0.0,0.0,2.702703,111
2,0,Ok lar... Joking wif u oni...,0.000000,0.0,0.0,6.896552,29
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,16.129032,0.0,0.0,6.451613,155
4,0,U dun say so early hor... U c already then say...,0.000000,0.0,0.0,4.081633,49
5,0,"Nah I don't think he goes to usf, he lives aro...",0.000000,0.0,0.0,3.278689,61


In [8]:
'''#I found this on the net :)
from keras.preprocessing.text import Tokenizer
class TextEncoder:
    def __init__(self):
        self.tokenizer = Tokenizer()
    
    def fit(self, texts):
        self.tokenizer.fit_on_texts(texts)
    
    def transform(self, texts):
        return self.tokenizer.texts_to_matrix(texts)
    
    def fit_transform(self, texts):
        self.fit(texts)
        return self.transform(texts)
    
    def dim(self):
        return len(self.tokenizer.word_index) + 1'''


'#I found this on the net :)\nfrom keras.preprocessing.text import Tokenizer\nclass TextEncoder:\n    def __init__(self):\n        self.tokenizer = Tokenizer()\n    \n    def fit(self, texts):\n        self.tokenizer.fit_on_texts(texts)\n    \n    def transform(self, texts):\n        return self.tokenizer.texts_to_matrix(texts)\n    \n    def fit_transform(self, texts):\n        self.fit(texts)\n        return self.transform(texts)\n    \n    def dim(self):\n        return len(self.tokenizer.word_index) + 1'

In [9]:
def isalpha(word):
    wrod = word.replace('.', '')
    return word.isalpha()

def clean_sms(text):
    text = text.lower()
    return (' '.join(filter(lambda x: isalpha(x) and x not in stopwords.words("english"), text.split()))).replace('.', '').split()

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
cv = CountVectorizer(strip_accents='ascii', min_df=5, analyzer=clean_sms)
data = pd.concat([data, pd.DataFrame(cv.fit_transform(data['text']).todense(), columns=cv.get_feature_names(), index=numpy.arange(1, cv.fit_transform(data['text']).todense().shape[0] + 1))], axis=1)
data.head(2)

,is spam,text,percentage digits,percentage question mark,percentage exclamation mark,percentage caps,length,abiola,able,abt,...,yet,ym,yo,yoga,yr,yrs,yun,yup,zed,ìï
1,0,"Go until jurong point, crazy.. Available only ...",0.0,0.0,0.0,2.702703,111,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Ok lar... Joking wif u oni...,0.0,0.0,0.0,6.896552,29,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y = data['is spam']
data = data.drop('is spam',axis=1)
data = data.drop('text',axis=1)
X = data

In [12]:
print(X.shape,y.shape)

(5572, 1287) (5572,)


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)


# MODELS

In [14]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf = clf.fit(x_train, y_train)

In [15]:
from sklearn.svm import SVC
clf1  = SVC(kernel='rbf',gamma=0.005,degree=5)
clf1 = clf1.fit(x_train, y_train)

In [208]:
#models
'''without using any features i got an f1_score of 0.92 using these models'''
#USED
model1 = Sequential()
model1.add(Dense(10,input_dim=1287))
model1.add(Dense(8,activation='relu'))
#model1.add(Dropout(0.05))
model1.add(Dense(6,activation='relu'))
model1.add(Dropout(0.01))
model1.add(Dense(4,activation='relu'))
#model1.add(Dropout(0.01))
model1.add(Dense(2,activation='relu'))
model1.add(Dense(1,activation='sigmoid'))


model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(x_train, y_train, shuffle=True, validation_split=.2, epochs=11, batch_size=30,verbose=2)
#overfitting after epoch 8 so keep 7
#accuracy: 98.09


Train on 3565 samples, validate on 892 samples
Epoch 1/11
 - 5s - loss: 0.6748 - acc: 0.8093 - val_loss: 0.6429 - val_acc: 0.9574
Epoch 2/11
 - 1s - loss: 0.6007 - acc: 0.9700 - val_loss: 0.5537 - val_acc: 0.9641
Epoch 3/11
 - 1s - loss: 0.5229 - acc: 0.9703 - val_loss: 0.4969 - val_acc: 0.9697
Epoch 4/11
 - 1s - loss: 0.4705 - acc: 0.9832 - val_loss: 0.4503 - val_acc: 0.9753
Epoch 5/11
 - 1s - loss: 0.4299 - acc: 0.9837 - val_loss: 0.4142 - val_acc: 0.9776
Epoch 6/11
 - 1s - loss: 0.3918 - acc: 0.9879 - val_loss: 0.3791 - val_acc: 0.9821
Epoch 7/11
 - 1s - loss: 0.3595 - acc: 0.9891 - val_loss: 0.3589 - val_acc: 0.9709
Epoch 8/11
 - 1s - loss: 0.3304 - acc: 0.9921 - val_loss: 0.3329 - val_acc: 0.9765
Epoch 9/11
 - 1s - loss: 0.3050 - acc: 0.9913 - val_loss: 0.3038 - val_acc: 0.9809
Epoch 10/11
 - 1s - loss: 0.2810 - acc: 0.9935 - val_loss: 0.2822 - val_acc: 0.9787
Epoch 11/11
 - 1s - loss: 0.2607 - acc: 0.9935 - val_loss: 0.2636 - val_acc: 0.9821


In [144]:
#NOT USED
model2 = Sequential()
model2.add(Dense(6,input_dim=1287))
model2.add(Dense(5,activation='relu'))
model2.add(Dropout(0.01))
model2.add(Dense(4,activation='relu'))
model2.add(Dense(3,activation='relu'))
model2.add(Dropout(0.01))
model2.add(Dense(2,activation='relu'))
model2.add(Dense(1,activation='sigmoid'))

model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(x_train, y_train, shuffle=True, validation_split=.2, epochs=10, batch_size=30,verbose=2)

Train on 3565 samples, validate on 892 samples
Epoch 1/10
 - 3s - loss: 0.4576 - acc: 0.8670 - val_loss: 0.3529 - val_acc: 0.8621
Epoch 2/10
 - 1s - loss: 0.2821 - acc: 0.8945 - val_loss: 0.2036 - val_acc: 0.9641
Epoch 3/10
 - 1s - loss: 0.1696 - acc: 0.9590 - val_loss: 0.1225 - val_acc: 0.9720
Epoch 4/10
 - 1s - loss: 0.1166 - acc: 0.9672 - val_loss: 0.0967 - val_acc: 0.9753
Epoch 5/10
 - 1s - loss: 0.1017 - acc: 0.9689 - val_loss: 0.0895 - val_acc: 0.9753
Epoch 6/10
 - 1s - loss: 0.0817 - acc: 0.9753 - val_loss: 0.0914 - val_acc: 0.9776
Epoch 7/10
 - 1s - loss: 0.0814 - acc: 0.9762 - val_loss: 0.0897 - val_acc: 0.9753
Epoch 8/10
 - 1s - loss: 0.0758 - acc: 0.9781 - val_loss: 0.1014 - val_acc: 0.9753
Epoch 9/10
 - 1s - loss: 0.0734 - acc: 0.9784 - val_loss: 0.1010 - val_acc: 0.9765
Epoch 10/10
 - 1s - loss: 0.0693 - acc: 0.9778 - val_loss: 0.0913 - val_acc: 0.9753


In [19]:
#NOT USED
model3 = Sequential()
model3.add(Dense(2,input_dim=1287))
model3.add(Dropout(0.4))
model3.add(Dense(1,activation='sigmoid'))

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(x_train, y_train, shuffle=True, validation_split=.2, epochs=10, batch_size=30,verbose=2)

Train on 3565 samples, validate on 892 samples
Epoch 1/10
 - 1s - loss: 0.9093 - acc: 0.7604 - val_loss: 0.3981 - val_acc: 0.9036
Epoch 2/10
 - 1s - loss: 0.3974 - acc: 0.9111 - val_loss: 0.3072 - val_acc: 0.9473
Epoch 3/10
 - 0s - loss: 0.3305 - acc: 0.9243 - val_loss: 0.2660 - val_acc: 0.9608
Epoch 4/10
 - 0s - loss: 0.2853 - acc: 0.9391 - val_loss: 0.2189 - val_acc: 0.9563
Epoch 5/10
 - 0s - loss: 0.2685 - acc: 0.9467 - val_loss: 0.1960 - val_acc: 0.9641
Epoch 6/10
 - 1s - loss: 0.2428 - acc: 0.9529 - val_loss: 0.1779 - val_acc: 0.9608
Epoch 7/10
 - 0s - loss: 0.2169 - acc: 0.9512 - val_loss: 0.1607 - val_acc: 0.9652
Epoch 8/10
 - 1s - loss: 0.1959 - acc: 0.9588 - val_loss: 0.1474 - val_acc: 0.9686
Epoch 9/10
 - 1s - loss: 0.1796 - acc: 0.9551 - val_loss: 0.1390 - val_acc: 0.9675
Epoch 10/10
 - 0s - loss: 0.1679 - acc: 0.9602 - val_loss: 0.1284 - val_acc: 0.9742


In [40]:
#NOT USED
model4 = Sequential()
model4.add(Dense(2,input_dim=1287))
model4.add(Dense(2,activation='relu'))
model4.add(Dropout(0.4))
model4.add(Dense(1,activation='sigmoid'))

model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model4.fit(x_train, y_train, shuffle=True, validation_split=.2, epochs=6, batch_size=35,verbose=2)

Train on 3565 samples, validate on 892 samples
Epoch 1/6
 - 1s - loss: 0.6756 - acc: 0.8662 - val_loss: 0.6586 - val_acc: 0.8621
Epoch 2/6
 - 0s - loss: 0.6421 - acc: 0.8670 - val_loss: 0.6273 - val_acc: 0.8621
Epoch 3/6
 - 0s - loss: 0.6119 - acc: 0.8670 - val_loss: 0.5993 - val_acc: 0.8621
Epoch 4/6
 - 0s - loss: 0.5851 - acc: 0.8670 - val_loss: 0.5742 - val_acc: 0.8621
Epoch 5/6
 - 1s - loss: 0.5612 - acc: 0.8670 - val_loss: 0.5521 - val_acc: 0.8621
Epoch 6/6
 - 0s - loss: 0.5399 - acc: 0.8670 - val_loss: 0.5326 - val_acc: 0.8621


In [173]:
#USED
model5 = Sequential()
model5.add(Dense(20,input_dim=1287))
model5.add(Dense(18,activation='relu'))
model5.add(Dropout(0.05))
model5.add(Dense(14,activation='relu'))
model5.add(Dense(10,activation='relu'))
model5.add(Dropout(0.05))
model5.add(Dense(8,activation='relu'))
model5.add(Dense(4,activation='relu'))
model5.add(Dropout(0.05))
model5.add(Dense(2,activation='relu'))
model5.add(Dense(1,activation='sigmoid'))

model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model5.fit(x_train, y_train, shuffle=True, validation_split=.2, epochs=10, batch_size=30,verbose=2)

Train on 3565 samples, validate on 892 samples
Epoch 1/10
 - 4s - loss: 0.6971 - acc: 0.8160 - val_loss: 0.6552 - val_acc: 0.8610
Epoch 2/10
 - 1s - loss: 0.6371 - acc: 0.8656 - val_loss: 0.6202 - val_acc: 0.8621
Epoch 3/10
 - 1s - loss: 0.5765 - acc: 0.9128 - val_loss: 0.5054 - val_acc: 0.9652
Epoch 4/10
 - 1s - loss: 0.4960 - acc: 0.9568 - val_loss: 0.4592 - val_acc: 0.9742
Epoch 5/10
 - 1s - loss: 0.4514 - acc: 0.9672 - val_loss: 0.4360 - val_acc: 0.9540
Epoch 6/10
 - 1s - loss: 0.4130 - acc: 0.9725 - val_loss: 0.3873 - val_acc: 0.9787
Epoch 7/10
 - 1s - loss: 0.3766 - acc: 0.9787 - val_loss: 0.3550 - val_acc: 0.9798
Epoch 8/10
 - 1s - loss: 0.3462 - acc: 0.9804 - val_loss: 0.3283 - val_acc: 0.9821
Epoch 9/10
 - 1s - loss: 0.3203 - acc: 0.9826 - val_loss: 0.3054 - val_acc: 0.9821
Epoch 10/10
 - 1s - loss: 0.2981 - acc: 0.9829 - val_loss: 0.2882 - val_acc: 0.9765


In [183]:
#USED
model8 = Sequential()
model8.add(Dense(20,input_dim=1287))
model8.add(Dense(18,activation='relu'))
model5.add(Dropout(0.05))
model8.add(Dense(14,activation='relu'))
model8.add(Dense(10,activation='relu'))
model8.add(Dropout(0.05))
model8.add(Dense(8,activation='relu'))
model8.add(Dense(4,activation='relu'))
model8.add(Dense(2,activation='relu'))
model8.add(Dense(1,activation='sigmoid'))

model8.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model8.fit(x_train, y_train, shuffle=True, validation_split=.2, epochs=10, batch_size=30,verbose=2)

Train on 3565 samples, validate on 892 samples
Epoch 1/10
 - 5s - loss: 0.6670 - acc: 0.8553 - val_loss: 0.6311 - val_acc: 0.9496
Epoch 2/10
 - 1s - loss: 0.5878 - acc: 0.9391 - val_loss: 0.5457 - val_acc: 0.9619
Epoch 3/10
 - 1s - loss: 0.5217 - acc: 0.9644 - val_loss: 0.4917 - val_acc: 0.9753
Epoch 4/10
 - 1s - loss: 0.4755 - acc: 0.9728 - val_loss: 0.4485 - val_acc: 0.9753
Epoch 5/10
 - 1s - loss: 0.4354 - acc: 0.9787 - val_loss: 0.4116 - val_acc: 0.9843
Epoch 6/10
 - 1s - loss: 0.3978 - acc: 0.9820 - val_loss: 0.3821 - val_acc: 0.9809
Epoch 7/10
 - 1s - loss: 0.3655 - acc: 0.9849 - val_loss: 0.3540 - val_acc: 0.9776
Epoch 8/10
 - 1s - loss: 0.3446 - acc: 0.9798 - val_loss: 0.3268 - val_acc: 0.9832
Epoch 9/10
 - 1s - loss: 0.3130 - acc: 0.9854 - val_loss: 0.3024 - val_acc: 0.9865
Epoch 10/10
 - 1s - loss: 0.2885 - acc: 0.9896 - val_loss: 0.2815 - val_acc: 0.9832


In [225]:
#trying to implement voting ensembling
y_pred1 = []
y_pred2 = []
y_pred8 = []
y_pred4 = []
y_pred5 = []
y_pred6 = []
y_pred7 = []


for i in model1.predict(x_test):
    if i>=0.5:
        y_pred1.append(1)
    else:
        y_pred1.append(0)
        
for i in model2.predict(x_test):
    if i>=0.5:
        y_pred2.append(1)
    else:
        y_pred2.append(0)

for i in model8.predict(x_test):
    if i>=0.5:
        y_pred8.append(1)
    else:
        y_pred8.append(0)

for i in model4.predict(x_test):
    if i>=0.5:
        y_pred4.append(1)
    else:
        y_pred4.append(0)

for i in model5.predict(x_test):
    if i>=0.5:
        y_pred5.append(1)
    else:
        y_pred5.append(0)
        
y_pred6 = clf.predict(x_test)
y_pred7 = clf1.predict(x_test)


y_pred = []
for i in range(len(y_pred1)):
    a,b,c = y_pred8[i],y_pred1[i],y_pred5[i]
    if (a+b+c)>=2:
        y_pred.append(1)
    else:
        y_pred.append(0)
        
y_actual = y_test   

In [230]:
from sklearn.metrics import f1_score
print('f1_score:',f1_score(y_actual,y_pred))

f1_score: 0.9764309764309764


In [194]:
from sklearn.metrics import classification_report
print(classification_report(y_actual,y_pred,target_names=['ham', 'spam']))

             precision    recall  f1-score   support

        ham       0.99      1.00      1.00       965
       spam       0.99      0.97      0.98       150

avg / total       0.99      0.99      0.99      1115



In [231]:
#model8.save('model8.h5')
#model1.save('model1.h5')
#model5.save('model5.h5')